In [217]:
import pandas as pd
import numpy as np
import ast
import pickle
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [218]:
df = pd.read_csv("New Scrap.csv", usecols=["tweet", "Sentimen"])
df.columns = ["tweet", "sentimen"]
df['sentimen'].value_counts(ascending=True)
#df.shape

Negatif    112
Positif    119
Netral     159
Name: sentimen, dtype: int64

In [219]:
def word_rekonizing(text):
  return word_tokenize(text)

df['tweet'] = df['tweet'].apply(word_rekonizing)
datatweet = df['tweet']
datatweet.head(10)

0    [jadi, pengen, urging, kalian, aja, yang, seha...
1    [erick, thohir, mengatakan, indonesia, memilik...
2    [pemerintah, terus, berupaya, keras, untuk, me...
3    [alamdulillah, pokoknya, jgn, pernah, nyopot, ...
4                    [lanjutkan, vaksin, merah, putih]
5    [pemerintahan, jokowi, kembali, menerima, kiri...
6    [kalau, kalian, di, pekanbaru, dan, eligible, ...
7    [enggak, usah, degan, vaksin, aman, kok, ku, j...
8    [erick, thohir, terima, 8, juta, bulk, vaksin,...
9    [amankan, kebutuhan, vaksin, pemerintah, kemba...
Name: tweet, dtype: object

In [220]:
from nltk.corpus import stopwords

In [221]:
list_stopwords = stopwords.words('indonesian')

list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                      'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'kali', 'wkwkwk', 'udh',
                       'oke', 'deh', 'tu', 'jg', 'lo', 'blm', 'yaa',
                       'pas', 'gitu', 'dapet', 'anjir', 'abg', 'eh',
                       'gue','dah','ni', 'tetep',' ya', 'bgt', 'dlm',
                       '2', 'udah','ku','gw', 'tp', 'wih', 'btw',
                       'sy', 'x', 'gk'])
#convert list to dictionary
list_stopwords = set(list_stopwords)

In [222]:
#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['tweet'] = df['tweet'].apply(stopwords_removal) 


print(df['tweet'].head(10))

0    [pengen, urging, sehat, divaksin, please, go, ...
1    [erick, thohir, indonesia, memiliki, 759, juta...
2    [pemerintah, berupaya, keras, menghadirkan, va...
3    [alamdulillah, pokoknya, nyopot, masker, gusss...
4                    [lanjutkan, vaksin, merah, putih]
5    [pemerintahan, jokowi, menerima, kiriman, vaks...
6    [pekanbaru, eligible, vaksin, mending, ikutan,...
7    [degan, vaksin, aman, suntik, glombang, bulang...
8    [erick, thohir, terima, 8, juta, bulk, vaksin,...
9    [amankan, kebutuhan, vaksin, pemerintah, datan...
Name: tweet, dtype: object


In [223]:
print("dimensi df :", df.shape)

dimensi df : (390, 2)


In [224]:
dfr = df.sample(frac=.50)
if (0.50*(len(df))== len(dfr)):
    print( "Cool")
    print(len(df), len(dfr))
#display
dfr

Cool
390 195


,tweet,sentimen
199,"[napa, anakku, mogok, makan, sakit, vaksin]",Netral
224,"[men, maaf, tolong, bikinin, vaksin, pake, dit...",Netral
143,"[vaksin, ngantri]",Netral
157,"[patuhi, prokes, vaksin]",Netral
315,"[harihari, cek, vaksin, lulus, ka, ndak, kadan...",Negatif
...,...,...
154,"[geng, anti, vaksin, percaya, antivax]",Netral
113,"[papa, mamaku, tergantung, vaksinnya, setau, v...",Positif
377,"[vaksin, giliran, menolak, nyinyir, suda, diva...",Negatif
357,"[cina, digugat, class, action, negara2, terdam...",Negatif


In [225]:
dfa= df.drop(dfr.index)
dfa.head()

,tweet,sentimen
0,"[pengen, urging, sehat, divaksin, please, go, ...",Positif
3,"[alamdulillah, pokoknya, nyopot, masker, gusss...",Positif
4,"[lanjutkan, vaksin, merah, putih]",Positif
10,"[buahahakalo, cairan, vaksin, mengandung, magn...",Positif
11,"[dosen, ngajar, keliaran, lingkungan, kampus, ...",Positif


In [226]:
x=df["tweet"]
y=df["sentimen"]
print(x)
print(y)

0      [pengen, urging, sehat, divaksin, please, go, ...
1      [erick, thohir, indonesia, memiliki, 759, juta...
2      [pemerintah, berupaya, keras, menghadirkan, va...
3      [alamdulillah, pokoknya, nyopot, masker, gusss...
4                      [lanjutkan, vaksin, merah, putih]
                             ...                        
385                               [vaksin, pengangguran]
386    [iya, lambat, vaksin, lokal, keburu, bubaran, ...
387    [masyarakat, takut, vaksin, walikota, kendari,...
388    [vaksinnya, ngga, vaksin, prewe, kampus, aing,...
389    [vaksin, prosedurnya, susah, males, emang, klu...
Name: tweet, Length: 390, dtype: object
0      Positif
1      Positif
2      Positif
3      Positif
4      Positif
        ...   
385    Negatif
386    Negatif
387    Negatif
388    Negatif
389    Negatif
Name: sentimen, Length: 390, dtype: object


In [227]:
#tf 
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

df['TF_dict'] = df["tweet"].apply(calc_TF)
df["TF_dict"]
#df["TF_dict"] = df['tweet'].apply(calc_TF)
#df["TF_dict"].head(10)

0      {'pengen': 0.043478260869565216, 'urging': 0.0...
1      {'erick': 0.043478260869565216, 'thohir': 0.04...
2      {'pemerintah': 0.03571428571428571, 'berupaya'...
3      {'alamdulillah': 0.09090909090909091, 'pokokny...
4      {'lanjutkan': 0.25, 'vaksin': 0.25, 'merah': 0...
                             ...                        
385                 {'vaksin': 0.5, 'pengangguran': 0.5}
386    {'iya': 0.14285714285714285, 'lambat': 0.14285...
387    {'masyarakat': 0.14285714285714285, 'takut': 0...
388    {'vaksinnya': 0.125, 'ngga': 0.125, 'vaksin': ...
389    {'vaksin': 0.06666666666666667, 'prosedurnya':...
Name: TF_dict, Length: 390, dtype: object

In [228]:
#idf
def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF


DF = calc_DF(df["TF_dict"])
#DF

In [229]:
n_document = len(df)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)


In [230]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
df["TF-IDF_dict"] = df["TF_dict"].apply(calc_TF_IDF)
df["TF-IDF_dict"].head()
#hasil_tfidf = dftrain["TF-IDF_dict"]
#hasil_tfidf

0    {'pengen': 0.21163193262850358, 'urging': 0.22...
1    {'erick': 0.16898718249755895, 'thohir': 0.174...
2    {'pemerintah': 0.10109473297123366, 'berupaya'...
3    {'alamdulillah': 0.4793635962330679, 'pokoknya...
4    {'lanjutkan': 1.3182498896409367, 'vaksin': 0....
Name: TF-IDF_dict, dtype: object

In [231]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(calc_DF(df["TF_dict"][df["sentimen"] == "Positif"]).items(), key=lambda kv: kv[1], reverse=True)[:18]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

df["TF_IDF_Vec"] = df["TF-IDF_dict"][df['sentimen'] == 'Positif'].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(df["TF_IDF_Vec"][df['sentimen'] == 'Positif'])

print("\nmatrix size : ", len(df["TF_IDF_Vec"][df['sentimen'] == 'Positif']))

print first row matrix TF_IDF_Vec Series

0      [0.0007874673888280914, 0.0, 0.0, 0.0, 0.0, 0....
1      [0.0007874673888280914, 0.1980232421747614, 0....
2      [0.001293696424503293, 0.08133097446463414, 0....
3      [0.0016465227220951004, 0.0, 0.0, 0.0, 0.0, 0....
4      [0.004527937485761526, 0.0, 0.0, 0.0, 0.0, 0.0...
                             ...                        
114    [0.006037249981015368, 0.0, 0.0, 0.0, 0.0, 0.0...
115    [0.004527937485761526, 0.0, 0.0, 0.0, 0.0, 0.0...
116    [0.0, 0.0, 0.0, 0.0, 0.0, 0.4513417001837017, ...
117    [0.0005659921857201907, 0.0, 0.0, 0.0, 0.0, 0....
118    [0.0010653970554733003, 0.0, 0.0, 0.0, 0.0, 0....
Name: TF_IDF_Vec, Length: 119, dtype: object

matrix size :  119


In [232]:
# Convert Series to List
TF_IDF_Vec_List = np.array(df["TF_IDF_Vec"][df['sentimen'] == 'Positif'].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
rankingPos = pd.DataFrame(data, columns=['term', 'rank'])
rankingPos.sort_values('rank', ascending=False)

,term,rank
1,juta,6.635892
3,dosis,5.631761
12,putih,5.174428
11,merah,5.076199
2,indonesia,4.554688
5,vaksinasi,3.722578
8,sinovac,3.676219
7,baku,3.395126
6,bahan,3.339360
9,pemerintah,3.220683


In [233]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(calc_DF(df["TF_dict"][df["sentimen"] == "Negatif"]).items(), key=lambda kv: kv[1], reverse=True)[:18]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

df["TF_IDF_Vec"] = df["TF-IDF_dict"][df['sentimen'] == 'Negatif'].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(df["TF_IDF_Vec"][df['sentimen'] == 'Negatif'])

print("\nmatrix size : ", len(df["TF_IDF_Vec"][df['sentimen'] == 'Negatif']))

print first row matrix TF_IDF_Vec Series

278    [0.0027864230681609392, 0.0, 0.0, 0.0, 0.23243...
279    [0.0006245431014843484, 0.0, 0.0, 0.0, 0.0, 0....
280    [0.0007874673888280914, 0.0, 0.0, 0.0, 0.0, 0....
281    [0.006037249981015368, 0.0, 0.0, 0.0, 0.0, 0.0...
282    [0.0027864230681609392, 0.0, 0.0, 0.0, 0.23243...
                             ...                        
385    [0.009055874971523051, 0.0, 0.0, 0.0, 0.0, 0.0...
386    [0.002587392849006586, 0.0, 0.0, 0.0, 0.0, 0.0...
387    [0.002587392849006586, 0.0, 0.0, 0.0, 0.0, 0.0...
388    [0.002263968742880763, 0.0, 0.0, 0.0, 0.0, 0.0...
389    [0.0012074499962030736, 0.0, 0.0, 0.0, 0.0, 0....
Name: TF_IDF_Vec, Length: 112, dtype: object

matrix size :  112


In [234]:
# Convert Series to List
TF_IDF_Vec_List = np.array(df["TF_IDF_Vec"][df['sentimen'] == 'Negatif'].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
rankingNeg = pd.DataFrame(data, columns=['term', 'rank'])
rankingNeg.sort_values('rank', ascending=False)

,term,rank
12,babi,4.797383
3,abis,3.775949
5,mati,3.693833
1,orang,3.348621
4,divaksin,2.983983
2,covid19,2.796762
8,takut,2.473329
16,anti,2.424865
7,covid,1.853582
15,disuntik,1.723753


In [235]:
# sort descending by value for DF dictionary 
sorted_DF = sorted(calc_DF(df["TF_dict"][df["sentimen"] == "Netral"]).items(), key=lambda kv: kv[1], reverse=True)[:18]

# Create a list of unique words from sorted dictionay `sorted_DF`
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)

    # For each unique word, if it is in the review, store its TF-IDF value.
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

df["TF_IDF_Vec"] = df["TF-IDF_dict"][df['sentimen'] == 'Netral'].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(df["TF_IDF_Vec"][df['sentimen'] == 'Netral'])

print("\nmatrix size : ", len(df["TF_IDF_Vec"][df['sentimen'] == 'Netral']))

print first row matrix TF_IDF_Vec Series

119    [0.0017249285660043906, 0.20566516660508305, 0...
120    [0.0007874673888280914, 0.0, 0.0, 0.0, 0.0, 0....
121    [0.0006468482122516465, 0.0, 0.0, 0.0, 0.10268...
122    [0.0016465227220951004, 0.0, 0.0, 0.0, 0.0, 0....
123    [0.0020699142792052687, 0.061699549981524916, ...
                             ...                        
273    [0.002587392849006586, 0.0, 0.1764027984040865...
274    [0.002263968742880763, 0.2699355311691715, 0.0...
275    [0.0020124166603384557, 0.0, 0.0, 0.0, 0.0, 0....
276    [0.004527937485761526, 0.0, 0.0, 0.0, 0.0, 0.0...
277    [0.0012074499962030736, 0.0, 0.0, 0.0, 0.0, 0....
Name: TF_IDF_Vec, Length: 159, dtype: object

matrix size :  159


In [236]:
# Convert Series to List
TF_IDF_Vec_List = np.array(df["TF_IDF_Vec"][df['sentimen'] == 'Netral'].to_list())

# Sum element vector in axis=0 
sums = TF_IDF_Vec_List.sum(axis=0)

data = []

for col, term in enumerate(unique_term):
    data.append((term, sums[col]))
    
rankingNet = pd.DataFrame(data, columns=['term', 'rank'])
rankingNet.sort_values('rank', ascending=False)

,term,rank
2,sinovac,4.245155
6,suntik,3.864634
8,astrazeneca,3.634559
1,indonesia,3.562652
11,jadwal,2.955273
10,daftar,2.881951
3,juta,2.874958
12,divaksin,2.853592
5,dosis,2.614517
16,habis,2.447058


In [237]:
feature = pd.concat([rankingPos,rankingNeg,rankingNet],axis=0,ignore_index=True)
feature.columns = ["term", "ranking"]
#feature.columns = ["termPositif", "rankingPositif", "termNegatif", "rankingNegatif", "termNetral", "rankingNetral",]
feature

,term,ranking
0,vaksin,0.312759
1,juta,6.635892
2,indonesia,4.554688
3,dosis,5.631761
4,covid19,3.211591
5,vaksinasi,3.722578
6,bahan,3.339360
7,baku,3.395126
8,sinovac,3.676219
9,pemerintah,3.220683


In [238]:
fiturr = feature.drop([10,13,41,49,50])
fiturr

,term,ranking
0,vaksin,0.312759
1,juta,6.635892
2,indonesia,4.554688
3,dosis,5.631761
4,covid19,3.211591
5,vaksinasi,3.722578
6,bahan,3.339360
7,baku,3.395126
8,sinovac,3.676219
9,pemerintah,3.220683


In [239]:
fitur = fiturr.drop_duplicates(subset='term', keep="first")
fitur["term"].shape

(35,)

In [240]:
#fitur = feature.drop_duplicates(subset='term', keep="first")
#fitur["term"].shape

In [241]:
#fitur2 = fitur.drop([3,7,10])
#fitur2

In [242]:
keyword_list = fitur["term"].values.tolist()
print (keyword_list)

['vaksin', 'juta', 'indonesia', 'dosis', 'covid19', 'vaksinasi', 'bahan', 'baku', 'sinovac', 'pemerintah', 'merah', 'putih', 'kebutuhan', 'menteri', 'kesehatan', 'semoga', 'orang', 'abis', 'divaksin', 'mati', 'covid', 'takut', 'demam', 'kena', 'badan', 'babi', 'habis', 'masuk', 'disuntik', 'anti', 'suntik', 'astrazeneca', 'daftar', 'jadwal', 'salah']


In [243]:
dseries = df["TF-IDF_dict"].apply(pd.Series)
new_dseries = dseries.fillna(0)
dtfidf = pd.DataFrame(new_dseries, columns=keyword_list)
dtfidf

,vaksin,juta,indonesia,dosis,covid19,vaksinasi,bahan,baku,sinovac,pemerintah,...,babi,habis,masuk,disuntik,anti,suntik,astrazeneca,daftar,jadwal,salah
0,0.000787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000787,0.198023,0.281672,0.103593,0.099012,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.001294,0.081331,0.077124,0.085094,0.081331,0.096716,0.104344,0.106086,0.088201,0.101095,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118825
3,0.001647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.004528,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,0.009056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
386,0.002587,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
387,0.002587,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
388,0.002264,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [244]:
x_train, x_test, y_train, y_test = train_test_split(dtfidf, y, test_size = 0.2)
#print(x_train)
#print(y_test.shape)

In [245]:
nb = MultinomialNB()
nb.fit(x_train,y_train)
prediksi = nb.predict(x_test)

In [246]:
from sklearn import metrics

In [247]:
print('\n akurasi data testing :', metrics.accuracy_score(y_test,prediksi))
print('\n', metrics.classification_report(y_test,prediksi))


 akurasi data testing : 0.5512820512820513

               precision    recall  f1-score   support

     Negatif       0.60      0.12      0.21        24
      Netral       0.52      0.86      0.65        37
     Positif       0.67      0.47      0.55        17

    accuracy                           0.55        78
   macro avg       0.60      0.49      0.47        78
weighted avg       0.58      0.55      0.49        78



In [248]:
print('\n akurasi data training :', metrics.accuracy_score(y_train,nb.predict(x_train)))
print('\n', metrics.classification_report(y_train,nb.predict(x_train)))


 akurasi data training : 0.5544871794871795

               precision    recall  f1-score   support

     Negatif       0.81      0.28      0.42        88
      Netral       0.47      0.88      0.61       122
     Positif       0.76      0.40      0.53       102

    accuracy                           0.55       312
   macro avg       0.68      0.52      0.52       312
weighted avg       0.66      0.55      0.53       312

